In [2]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression,Ridge,Lasso
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score,train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import r2_score,mean_squared_error
from xgboost import XGBRegressor

In [3]:
data=pd.read_csv(r"C:\Users\RAMESWAR BISOYI\Downloads\processed.csv")

In [4]:
data

,onehotencoder__Item_Type_Baking Goods,onehotencoder__Item_Type_Breads,onehotencoder__Item_Type_Breakfast,onehotencoder__Item_Type_Canned,onehotencoder__Item_Type_Dairy,onehotencoder__Item_Type_Frozen Foods,onehotencoder__Item_Type_Fruits and Vegetables,onehotencoder__Item_Type_Hard Drinks,onehotencoder__Item_Type_Health and Hygiene,onehotencoder__Item_Type_Household,...,Outlet_Age_Category_Old,Outlet_Age_Category_Very Old,Yearly_Sales_Trend,Item_Sales_Rank,scaled_remainder__Item_Weight,scaled_remainder__Item_Visibility,scaled_remainder__Item_MRP,scaled_remainder__Item_Outlet_Sales,scaled_remainder__Outlet_Age,log_transformed_sales
0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,2348.354635,1447.5,-0.828359,-1.170199,1.747351,0.971666,-0.139533,0.678879
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1995.498739,7420.5,-1.625269,-1.099096,-1.488936,0.000000,-1.334024,0.000000
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,2348.354635,3674.5,1.104972,-1.154513,0.010040,0.000000,-0.139533,0.000000
3,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,339.351662,6655.5,1.505785,-0.067981,0.660011,0.000000,-0.020083,0.000000
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,1.0,2298.995256,6033.0,-0.915595,-0.067981,-1.399138,0.000000,1.293858,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8518,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,2298.995256,2593.0,-1.402464,-0.273717,1.180714,0.382845,1.293858,0.324143
8519,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,2192.384798,7132.0,-1.045270,-0.489408,-0.527270,0.000000,-0.497880,0.000000
8520,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,2438.841866,5593.5,-0.521856,-0.749006,-0.897155,0.000000,-0.736779,0.000000
8521,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1995.498739,4150.0,-1.321123,1.672526,-0.607941,0.000000,-1.334024,0.000000


In [8]:
import numpy as np
import pandas as pd

# Assuming `data` is your DataFrame
# Fill NaN and replace negatives in the columns you're applying log transformations to
data['scaled_remainder__Item_Visibility'] = data['scaled_remainder__Item_Visibility'].clip(lower=0)
data['scaled_remainder__Item_Outlet_Sales'] = data['scaled_remainder__Item_Outlet_Sales'].clip(lower=0)

# Apply log transformation after handling negatives and NaNs
data['Item_Visibility_Log'] = np.log1p(data['scaled_remainder__Item_Visibility'])
data['Item_Outlet_Sales_Log'] = np.log1p(data['scaled_remainder__Item_Outlet_Sales'])

# Define IQR outlier handling function
def handle_outliers_iqr(column):
    Q1 = data[column].quantile(0.25)
    Q3 = data[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    return data[column].clip(lower=lower_bound, upper=upper_bound)

# Handle outliers for log-transformed columns
data['Item_Visibility_Log'] = handle_outliers_iqr('Item_Visibility_Log')
data['Item_Outlet_Sales_Log'] = handle_outliers_iqr('Item_Outlet_Sales_Log')


In [10]:
# Splitting features and target variable
X = data.drop(columns=['scaled_remainder__Item_Outlet_Sales', 'Item_Outlet_Sales_Log'])
y = data['Item_Outlet_Sales_Log']

In [12]:
# Splitting into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [14]:
# Train a simple Random Forest model to get feature importances
rf = RandomForestRegressor(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)

RandomForestRegressor(random_state=42)

In [15]:
# Get feature importances and select top features
importances = rf.feature_importances_
important_features = X.columns[np.argsort(importances)[-10:]]  # Select top 10 features

In [16]:
# Reducing X to only important features
X_train_important = X_train[important_features]
X_test_important = X_test[important_features]

In [20]:
print(X_train.shape,X_train_important.shape)

(6818, 47) (6818, 10)


In [22]:
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.model_selection import cross_val_score, GridSearchCV
import numpy as np

# Initialize models with default parameters
models = {
    "Linear Regression": LinearRegression(),
    "Ridge Regression": Ridge(alpha=1.0, random_state=42),
    "Lasso Regression": Lasso(alpha=0.1, random_state=42),
    "Random Forest": RandomForestRegressor(random_state=42),
    "XGBoost": XGBRegressor(objective='reg:squarederror', random_state=42)
}

# Define parameter grids for hyperparameter tuning
param_grids = {
    "Random Forest": {
        'n_estimators': [100, 200, 300],
        'max_depth': [None, 10, 20, 30],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4]
    },
    "XGBoost": {
        'n_estimators': [100, 200, 300],
        'max_depth': [3, 5, 7, 10],
        'learning_rate': [0.01, 0.1, 0.2],
        'subsample': [0.7, 0.8, 1.0],
        'colsample_bytree': [0.7, 0.8, 1.0]
    }
}

# Function to evaluate models using cross-validation and hyperparameter tuning
def evaluate_models(models, param_grids, X, y):
    results = {}
    for name, model in models.items():
        print(f"\nEvaluating {name}...")
        # Perform grid search for models with parameter grids
        if name in param_grids:
            grid_search = GridSearchCV(model, param_grids[name], cv=10, scoring='r2', n_jobs=-1)
            grid_search.fit(X, y)
            best_model = grid_search.best_estimator_
            cv_scores = cross_val_score(best_model, X, y, cv=10, scoring='r2')
            print(f"Best parameters for {name}: {grid_search.best_params_}")
        else:
            # Use cross-validation with default parameters
            cv_scores = cross_val_score(model, X, y, cv=10, scoring='r2')
        
        results[name] = {
            "Mean R²": cv_scores.mean(),
            "Standard Deviation": cv_scores.std()
        }
    return results

# Evaluate all models on the important features
model_results = evaluate_models(models, param_grids, X_train_important, y_train)

# Display results
for model, metrics in model_results.items():
    print(f"{model}: Mean R² = {metrics['Mean R²']:.4f}, Std Dev = {metrics['Standard Deviation']:.4f}")

# Choose the best model based on the highest mean R² score
best_model_name = max(model_results, key=lambda x: model_results[x]["Mean R²"])
print(f"\nBest Model: {best_model_name}")



Evaluating Linear Regression...

Evaluating Ridge Regression...

Evaluating Lasso Regression...

Evaluating Random Forest...
Best parameters for Random Forest: {'max_depth': 20, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 300}

Evaluating XGBoost...
Best parameters for XGBoost: {'colsample_bytree': 1.0, 'learning_rate': 0.1, 'max_depth': 7, 'n_estimators': 200, 'subsample': 1.0}
Linear Regression: Mean R² = 0.9987, Std Dev = 0.0001
Ridge Regression: Mean R² = 0.9986, Std Dev = 0.0001
Lasso Regression: Mean R² = 0.9310, Std Dev = 0.0031
Random Forest: Mean R² = 1.0000, Std Dev = 0.0000
XGBoost: Mean R² = 1.0000, Std Dev = 0.0000

Best Model: Random Forest


In [22]:
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.model_selection import cross_val_score, RepeatedKFold, StratifiedKFold
import numpy as np

# Initialize models with default parameters
models = {
    "Linear Regression": LinearRegression(),
    "Ridge Regression": Ridge(alpha=1.0, random_state=42),
    "Lasso Regression": Lasso(alpha=0.1, random_state=42),
    "Random Forest": RandomForestRegressor(n_estimators=100, random_state=42),
    "XGBoost": XGBRegressor(objective='reg:squarederror', n_estimators=100, random_state=42)
}

# Function to evaluate models using advanced cross-validation
def evaluate_models(models, X, y):
    results = {}
    # Use RepeatedKFold with stratified targets
    rkf = RepeatedKFold(n_splits=10, n_repeats=5, random_state=42)
    
    # Bin the target variable for Stratified K-Fold
    y_binned = np.digitize(y, bins=np.linspace(y.min(), y.max(), 10))
    
    for name, model in models.items():
        # Perform repeated stratified cross-validation
        cv_scores = cross_val_score(model, X, y, cv=rkf, scoring='r2')
        
        # Store mean R² and standard deviation for the model
        results[name] = {
            "Mean R²": cv_scores.mean(),
            "Standard Deviation": cv_scores.std()
        }
    return results

# Evaluate all models on the important features
model_results = evaluate_models(models, X_train_important, y_train)

# Display results
for model, metrics in model_results.items():
    print(f"{model}: Mean R² = {metrics['Mean R²']:.4f}, Std Dev = {metrics['Standard Deviation']:.4f}")

# Choose the best model based on the highest mean R² score, ensuring no exact perfect scores
best_model_name = max(
    (m for m, s in model_results.items() if s["Mean R²"] < 0.99), 
    key=lambda x: model_results[x]["Mean R²"]
)
print(f"\nBest Model: {best_model_name}")


Linear Regression: Mean R² = 0.9987, Std Dev = 0.0001
Ridge Regression: Mean R² = 0.9986, Std Dev = 0.0001
Lasso Regression: Mean R² = 0.9310, Std Dev = 0.0033
Random Forest: Mean R² = 1.0000, Std Dev = 0.0000
XGBoost: Mean R² = 1.0000, Std Dev = 0.0000

Best Model: Lasso Regression


In [ ]:
from sklearn.model_selection import GridSearchCV
from xgboost import XGBRegressor

# Define the parameter grid for XGBoost
param_grid = {
        'n_estimators': [100, 200, 300],
        'max_depth': [3, 5, 7, 10],
        'learning_rate': [0.01, 0.1, 0.2],
        'subsample': [0.7, 0.8, 1.0],
        'colsample_bytree': [0.7, 0.8, 1.0]
    }

# Initialize the XGBoost Regressor
xgb = XGBRegressor(objective='reg:squarederror', random_state=42)

# Use GridSearchCV to find the best hyperparameters
grid_search = GridSearchCV(
    estimator=xgb,
    param_grid=param_grid,
    scoring='r2',
    cv=5,
    n_jobs=-1
)

# Fit the model to the training data
grid_search.fit(X_train_important, y_train)


In [ ]:
best_xgb_model = grid_search.best_estimator_
best_params = grid_search.best_params_

In [ ]:
# Evaluate the best XGBoost model on the test set
y_pred_best_xgb = best_xgb_model.predict(X_test_important)
mse_best_xgb = mean_squared_error(y_test, y_pred_best_xgb)
r2_best_xgb = r2_score(y_test, y_pred_best_xgb)

# Output the results
print("Best Hyperparameters:", best_params)
print("Mean Squared Error (Best XGBoost):", mse_best_xgb)
print("R² Score (Best XGBoost):", r2_best_xgb)
